# Import the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Efficient Frontier

Investors have different appetites for risk

Some favour safer returns at the expense of less change of exceptional returns.

Portfolio theory is that some risk can be mitigated by mixing ecurities together

Can allow investors to earn higher rate of return whilst reducing risk

## Load in some Data

1 years worth

Approx 252 days

In [2]:
stocks = 'AAPL GLD'.split()
df = yf.download(tickers=stocks, start='2021-01-01')['Adj Close'].copy()



[*********************100%***********************]  2 of 2 completed


In [5]:
df = df.iloc[-253:]

In [6]:
df_vol = pd.DataFrame()

for stock in stocks:
    if stock not in df_vol:
        df_vol[stock] = np.log(df[stock]).diff() 

In [7]:
df_vol


,AAPL,GLD
Date,,
2021-08-09,NaN,NaN
2021-08-10,-0.003360,0.000309
2021-08-11,0.001784,0.013691
2021-08-12,0.020560,0.000244
2021-08-13,0.001409,0.014224
...,...,...
2022-08-03,0.037534,0.002435
2022-08-04,-0.001928,0.016405
2022-08-05,-0.001390,-0.011310


## Annualize variances

Multiply daily variance by 252

In [12]:
1 - 165/182

0.09340659340659341

In [8]:
var_aapl = df_vol['AAPL'].var() * 252
var_gld = df_vol['GLD'].var() * 252
print(var_aapl)
print(var_gld)

0.09269675220237121
0.019194420269890648


## Construct a Portfolio

Weights - 90% apple, 10% gold

Expected Returns -  (Apple 14%), Gold(7%)

In [18]:
w_aapl =  .9
w_gld = 1 - w_aapl
exp_aapl = .14
exp_gld = .07

In [19]:
exp = w_aapl * exp_aapl + w_gld * exp_gld
exp

0.13300000000000003

### Anualize the covariance

In [15]:
np.cov(df_vol['AAPL'][1:], df_vol['GLD'][1:])[1,0]

-1.4286516868215366e-05

In [16]:
cov = np.cov(df_vol['AAPL'][1:], df_vol['GLD'][1:])[0,1] * 252
cov

-0.003600202250790272

### Calculate std of Portfolio

Based on weights (90% & 10%)

$ \large std_{port} = \sqrt{var_{aapl}.(weight_{aapl})^2 + var_{gld}.(weight_{gld})^2 + 2. cov.weight_{aapl}.weight_{gld} }$

In [17]:
port_std = np.sqrt(var_aapl * w_aapl **2 + var_gld * w_gld ** 2 + 2 * cov * w_aapl * w_gld)
port_std

0.27318176564602065

## Analyse Results

With 90% Apple & 10% gold

- Expect return is 13.3%
- Expected volatility is 36.7%

Might consider this too high a risk

## Repeat

But with difference weights

e.g. weights of 0%, 5%, 10%, 155, ... 100%

In [20]:
df_effic = pd.DataFrame({'weight_aapl':np.zeros(21), 'exp_ret': np.zeros(21), 'std': np.zeros(21)})

df_effic

,weight_aapl,exp_ret,std
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [24]:
df_effic = pd.DataFrame({'weight_aapl':np.zeros(21), 'exp_ret': np.zeros(21), 'std': np.zeros(21)})

w_aapl = 0.0
for weight in range(21):
    df_effic['weight_aapl'][weight] = w_aapl
    df_effic['exp_ret'][weight] = w_aapl * exp_aapl + (1-w_aapl) * exp_gld
    #df_effic['std'][weight] = np.sqrt(var_aapl * w_aapl ** 2 + var_gld * (1-w_aapl) ** 2 + cov * w_aapl * (1 - w_aapl))
    w_aapl = w_aapl + .05

df_effic

,weight_aapl,exp_ret,std
0,0.00,0.0700,0.0
1,0.05,0.0735,0.0
2,0.10,0.0770,0.0
3,0.15,0.0805,0.0
4,0.20,0.0840,0.0
5,0.25,0.0875,0.0
6,0.30,0.0910,0.0
7,0.35,0.0945,0.0
8,0.40,0.0980,0.0
9,0.45,0.1015,0.0


In [ ]:
with plt.style.context('seaborn'):
    plt.scatter(df_effic['std'],df_effic['exp_ret'])